In [1]:
import numpy as np
from tqdm import tqdm
from solver.qp import qp
import pandas as pd
import torch
import gzip
import pickle
import warnings
from generate_instances import generate_softmarginsvm, generate_markowitz_portfolio_optimization, generate_convex_function_to_data, generate_regressor_selection

### Soft margin Support Vector Machine

In [30]:
def surrogate_gen():
    NUM = 500 # Number of points
    DIM = 2 # Dimension of the points
    C = 10 # regularization parameter (hardness of the margin)
    
    
    # 2-D mean of ones
    M1 = np.ones((DIM,))
    # 2-D mean of threes
    M2 = 3 * np.ones((DIM,))
    # 2-D covariance of 0.3
    C1 = np.diag(0.3 * np.ones((DIM,)))
    # 2-D covariance of 0.2
    C2 = np.diag(0.2 * np.ones((DIM,)))

    # generate 50 points from gaussian 1
    x1 = np.random.multivariate_normal(M1, C1, NUM) #generate gaussian
    # labels
    y1 = np.ones((x1.shape[0],))
    # generate 50 points from gaussian 2
    x2 = np.random.multivariate_normal(M2, C2, NUM)
    y2 = -np.ones((x2.shape[0],))
    # join
    X = np.concatenate((x1, x2), axis = 0)
    y = np.concatenate((y1, y2), axis = 0)

    Q, q, G, h, A, b, S = generate_softmarginsvm(y, X, C)    
    return Q, q, G, h, A, b, S

### Markowitz portfolio optimization

In [28]:
def surrogate_gen():
    np.random.seed(1)
    n_obs = 252
    n_assets = 4
    artificial_returns = np.random.randn(n_obs, n_assets) + 0.05 # minimum expected return
    returns = pd.DataFrame(artificial_returns)
    pbar = np.array(returns.mean()).reshape(1, -1)
    r_min = pbar.mean() # minimum return threshold
    Q, q, G, h, A, b, S = generate_markowitz_portfolio_optimization(artificial_returns, r_min, pbar)
    return Q, q, G, h, A, b, S

### Fiting convex function to given data

In [23]:
def surrogate_gen():
    
    # Generate 'u' values (from -2 to 2) for the independent variable
    NUM = 4
    u_values = np.linspace(-2.0, 2.0, num=NUM).reshape(-1, 1)
    Q, q, G, h, A, b, S = generate_convex_function_to_data(u_values)
    return Q, q, G, h, A, b, S

### Regressor selection problem

In [26]:
def surrogate_gen():
    
    # Set dimensions
    m = 10  # number of rows for A
    n = 20  # number of columns for A
    nopts = 100 # range of alpha values
    alpha_index = np.random.randint(0, nopts) # Select an alpha value from the range nopts
    Q, q, G, h, A, b, S = generate_regressor_selection(alpha_index, m, n)
    
    return Q, q, G, h, A, b, S

### create

In [31]:
import os

# directory = f'../../../../work/log1/darius.weber/Quadratic_Programming_Datasets/raw'
directory = f'Quadratic_Programming_Datasets/raw'
os.makedirs(directory, exist_ok=True)
warnings.filterwarnings("error")

ips = []
pkg_idx = 0 #identifier index
success_cnt = 0
fail_cnt = 0
max_ipm_steps = 0


max_iter = 100
num = 10

for i in tqdm(range(max_iter)):
    Q, q, G, h, A, b, S = surrogate_gen()
    try:
        res = qp(Q, q, G, h, A, b, callback=lambda res: res)
        max_ipm_steps = max(max_ipm_steps, res['iterations'])
        print("status",res['status'])
    except Exception as e:
        fail_cnt += 1
        warnings.warn(f'Optimization failed with error: {str(e)}')
        continue
    else:
        if res['status'] == 'optimal': #np.isnan(res.fun)?
            ips.append((torch.from_numpy(np.array(Q)).to(torch.float), torch.from_numpy(np.array(q)).to(torch.float), torch.from_numpy(np.array(G)).to(torch.float), torch.from_numpy(np.array(h)).to(torch.float), torch.from_numpy(np.array(A)).to(torch.float), torch.from_numpy(np.array(b)).to(torch.float), torch.from_numpy(np.array(S)).to(torch.float), res))
            success_cnt += 1
            
    if len(ips) >= 1000 or success_cnt == num:
        print("success")
        ips = [ip + (max_ipm_steps,) for ip in ips]
        with gzip.open(f'{directory}/instance_{pkg_idx}.pkl.gz', "wb") as file:
            pickle.dump(ips, file)
            print(f'Package {pkg_idx} saved')
            pkg_idx += 1

        ips = []
    if success_cnt >= num:
        break
warnings.resetwarnings()

  0%|          | 0/100 [00:00<?, ?it/s]

     pcost       dcost       gap    pres   dres
 0: -6.0380e+02 -7.0362e+04  2e+05  9e-01  9e-13
 1: -2.4600e+02 -1.8026e+04  3e+04  8e-02  5e-13
 2: -1.4781e+01 -3.4578e+03  5e+03  1e-02  3e-13
 3:  3.3993e+01 -1.2021e+03  2e+03  4e-03  1e-13
 4:  3.6605e+01 -5.6148e+02  8e+02  2e-03  7e-14
 5:  1.7819e+01 -2.4970e+02  3e+02  6e-04  4e-14
 6:  9.4349e+00 -1.4416e+02  2e+02  2e-04  5e-14
 7:  6.2389e+00 -1.3277e+02  2e+02  1e-04  6e-14
 8: -1.6065e+00 -8.9845e+01  9e+01  3e-05  6e-14
 9: -3.6602e+00 -8.2440e+01  8e+01  2e-05  6e-14
10: -1.1402e+01 -7.3282e+01  6e+01  1e-05  7e-14
11: -9.1571e+00 -6.9661e+01  6e+01  1e-05  7e-14
12: -2.2037e+01 -4.6511e+01  2e+01  3e-06  7e-14
13: -2.5386e+01 -3.9003e+01  1e+01  1e-06  6e-14
14: -2.5642e+01 -3.9078e+01  1e+01  1e-06  6e-14
15: -2.9632e+01 -3.1893e+01  2e+00  2e-07  7e-14
16: -3.0499e+01 -3.0531e+01  3e-02  2e-09  8e-14
17: -3.0512e+01 -3.0513e+01  3e-04  2e-11  7e-14


  1%|          | 1/100 [00:02<04:00,  2.43s/it]

18: -3.0513e+01 -3.0513e+01  3e-06  2e-13  8e-14
Optimal solution found.
status optimal
     pcost       dcost       gap    pres   dres
 0: -5.8864e+02 -6.0334e+04  2e+05  8e-01  8e-13
 1: -2.3383e+02 -1.2797e+04  2e+04  6e-02  6e-13
 2: -6.2709e+01 -2.1756e+03  3e+03  9e-03  3e-13
 3: -4.3915e+01 -4.9811e+02  6e+02  1e-03  1e-13
 4: -5.7853e+01 -4.1456e+02  4e+02  8e-04  1e-13
 5: -7.1475e+01 -3.1104e+02  3e+02  3e-04  1e-13
 6: -8.0035e+01 -2.1896e+02  1e+02  6e-16  1e-13
 7: -9.4082e+01 -1.7728e+02  8e+01  5e-15  1e-13
 8: -1.0355e+02 -1.5349e+02  5e+01  2e-15  1e-13
 9: -1.0881e+02 -1.4071e+02  3e+01  1e-14  1e-13
10: -1.1108e+02 -1.3841e+02  3e+01  1e-14  1e-13
11: -1.1491e+02 -1.3284e+02  2e+01  3e-15  1e-13
12: -1.1509e+02 -1.3060e+02  2e+01  3e-14  1e-13
13: -1.1640e+02 -1.2870e+02  1e+01  3e-15  1e-13
14: -1.1959e+02 -1.2416e+02  5e+00  4e-15  2e-13
15: -1.2122e+02 -1.2207e+02  8e-01  3e-14  2e-13
16: -1.2126e+02 -1.2203e+02  8e-01  2e-14  2e-13
17: -1.2159e+02 -1.2162e+02  3e

  2%|▏         | 2/100 [00:04<03:42,  2.27s/it]

...

19: -8.2160e+01 -8.2160e+01  2e-05  4e-14  1e-13
Optimal solution found.
status optimal
     pcost       dcost       gap    pres   dres
 0: -5.9981e+02 -7.0342e+04  2e+05  9e-01  7e-13
 1: -2.4230e+02 -1.8053e+04  3e+04  8e-02  6e-13
 2: -2.3912e+00 -2.9614e+03  5e+03  1e-02  3e-13
 3:  3.5673e+01 -1.0688e+03  2e+03  4e-03  2e-13
 4:  3.9790e+01 -4.7164e+02  6e+02  1e-03  7e-14
 5:  1.8583e+01 -1.3658e+02  2e+02  1e-04  4e-14
 6:  1.7935e+01 -1.2992e+02  2e+02  9e-05  4e-14
 7:  3.7676e+00 -9.1652e+01  1e+02  4e-05  5e-14
 8: -5.5941e+00 -6.0982e+01  6e+01  2e-05  5e-14
 9: -5.6725e+00 -5.8173e+01  5e+01  1e-05  5e-14
10: -1.4350e+01 -4.1825e+01  3e+01  2e-06  6e-14
11: -2.2657e+01 -2.7296e+01  5e+00  2e-07  7e-14
12: -2.3217e+01 -2.6009e+01  3e+00  3e-08  7e-14
13: -2.4392e+01 -2.4723e+01  3e-01  2e-09  7e-14


  9%|▉         | 9/100 [00:19<03:30,  2.32s/it]

14: -2.4549e+01 -2.4556e+01  7e-03  5e-11  8e-14
15: -2.4553e+01 -2.4553e+01  8e-05  5e-13  8e-14
16: -2.4553e+01 -2.4553e+01  8e-07  3e-14  7e-14
Optimal solution found.
status optimal
     pcost       dcost       gap    pres   dres
 0: -6.4231e+02 -8.4545e+04  3e+05  1e+00  7e-13
 1: -3.0202e+02 -2.6105e+04  4e+04  1e-01  6e-13
 2: -4.3350e+01 -4.8560e+03  7e+03  2e-02  4e-13
 3:  1.4423e+01 -1.7178e+03  2e+03  5e-03  2e-13
 4:  1.7305e+01 -8.8877e+02  1e+03  2e-03  1e-13
 5: -2.0173e+01 -3.2601e+02  3e+02  5e-04  1e-13
 6: -2.9935e+01 -2.4233e+02  2e+02  2e-04  8e-14
 7: -3.7787e+01 -2.4187e+02  2e+02  2e-04  1e-13
 8: -4.3487e+01 -2.0996e+02  2e+02  1e-04  1e-13
 9: -5.7381e+01 -1.3236e+02  7e+01  1e-14  1e-13
10: -6.0124e+01 -1.3465e+02  7e+01  9e-15  1e-13
11: -6.1777e+01 -1.1641e+02  5e+01  2e-14  1e-13
12: -6.2324e+01 -1.1675e+02  5e+01  1e-14  1e-13
13: -7.5833e+01 -9.9955e+01  2e+01  2e-14  1e-13
14: -8.4071e+01 -8.6271e+01  2e+00  2e-14  1e-13
15: -8.4734e+01 -8.5154e+01  4e

  9%|▉         | 9/100 [00:29<05:00,  3.30s/it]

Package 0 saved


In [ ]:
# Open and load the .pkl.gz file
with gzip.open(f'Quadratic_Programming_Datasets/raw/instance_{pkg_idx-1}.pkl.gz', 'rb') as f:
    data = pickle.load(f)

# Now `data` contains the contents of the pickle file
print(data)